In [1]:
# Import Plant

from model.plant import Plant, RearrangmentPlantV1, RearrangmentPlantV2
from model.tools import SystemSpecification
from model import StationNameType
from model import Vector

In [2]:
# Data generation

model_file_path = "../model.yaml"

model_file = open(model_file_path, "r")

spec = SystemSpecification(model_stream=model_file)

plant = Plant(spec)

plant.import_config([
    (Vector(2,0), "InOut"),
    (Vector(2,1), "Robot1"),
    (Vector(2,2), "Press"),
    (Vector(1, 1), "PartsStorage"),
    (Vector(1,2), "Robot2"),

])

print("Original plant")
plant.print()

objective_plant = Plant(spec)

objective_plant.import_config([
    (Vector(2,0), "InOut"),
    (Vector(2,1), "Robot1"),
    (Vector(1,1), "Press"),
    (Vector(2, 2), "PartsStorage"),
    (Vector(1,2), "Robot2"),

])

print("Objective plant")
objective_plant.print()

manipulated_plant = Plant(spec)

StorageBufferNameType = str
sequence_list: list[tuple[StationNameType, Vector | StorageBufferNameType, Vector | StorageBufferNameType]] = []

Original plant
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|                 |        0        |        1        |        2        |        3        |        4        |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|        0        |       None      |       None      |      InOut      |       None      |       None      |
|        1        |       None      |   PartsStorage  |      Robot1     |       None      |       None      |
|        2        |       None      |      Robot2     |      Press      |       None      |       None      |
|        3        |       None      |       None      |       None      |       None      |       None      |
|        4        |       None      |       None      |       None      |       None      |       None      |
+-----------------+-----------------+-----------------+-----------------+-----------------+--------------

In [3]:
# Print results function
def print_results():
    # Show the final plant and the objective plant
    print("Manipulated Plant final version")
    manipulated_plant.print()

    print("Sequence List")
    for item in sequence_list:
        print(item)


In [4]:
# Version 1

# Compare the two plants, create a 2D array of the differences

manipulated_plant = RearrangmentPlantV1(spec)

manipulated_plant.import_config(plant.export_config())


equal = plant.compare(objective_plant)


# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position


for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            for i in range(spec.model.stations.grid.size.x - 1,  x - 1, -1):
                if manipulated_plant.is_empty_coord(i, y):
                    continue
                result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
                sequence_list.append((result[0], Vector(i, y), f"S{result[1]}"))


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.compare(objective_plant)

for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            objective_station_name = objective_plant.get_location_coordinates(x, y).name
            result = manipulated_plant.move_station_from_buffer_to_coord(objective_station_name, x, y)
            sequence_list.append((objective_station_name, f"S{result}", Vector(x, y)))

In [5]:
# Version 2

# The next version should come with limited storage capacity

# Compare the two plants, create a 2D array of the differences

StorageBufferNameType = str

manipulated_plant = RearrangmentPlantV2(spec, 3)

manipulated_plant.import_config(plant.export_config())

sequence_list: list[tuple[StationNameType, Vector | StorageBufferNameType, Vector | StorageBufferNameType]] = []

equal = plant.compare(objective_plant)


# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position


for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            for i in range(spec.model.stations.grid.size.x - 1,  x - 1, -1):
                if manipulated_plant.is_empty_coord(i, y):
                    continue
                result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
                sequence_list.append((result[0], Vector(i, y), f"S{result[1]}"))


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.compare(objective_plant)

for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            objective_station_name = objective_plant.get_location_coordinates(x, y).name
            result = manipulated_plant.move_station_from_buffer_to_coord(objective_station_name, x, y)
            sequence_list.append((objective_station_name, f"S{result}", Vector(x, y)))

print_results()

Manipulated Plant final version
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|                 |        0        |        1        |        2        |        3        |        4        |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|        0        |       None      |       None      |      InOut      |       None      |       None      |
|        1        |       None      |      Press      |      Robot1     |       None      |       None      |
|        2        |       None      |      Robot2     |   PartsStorage  |       None      |       None      |
|        3        |       None      |       None      |       None      |       None      |       None      |
|        4        |       None      |       None      |       None      |       None      |       None      |
+-----------------+-----------------+-----------------+-----------------+---------------

The next version should be able to simplify the sequence list, by merging consecutive moves of the same station

It can be done something similar in three ways:

 - By post-processing the output of the V2 version, merging consecutive moves of the same station. This is the simplest way, but it is not the most efficient one.
 - By, at any time, when a station is moved to the storage buffer, check if can be fitted directly in other row, by brute force. This way is also simple, but will slow down the algorithm.
 - Another option is, when the last station from a row is moved to the storage buffer and the remaining stations are in right place, save the station name and position in a list named ready_to_install. Later, whenever a station is moved to the storage buffer, check if it is in the ready_to_install list, then move it to the right place. 

In [6]:
# Version 3

# Compare the two plants, create a 2D array of the differences

StorageBufferNameType = str

manipulated_plant = RearrangmentPlantV2(spec, 3)

manipulated_plant.import_config(plant.export_config())

sequence_list: list[tuple[StationNameType, Vector | StorageBufferNameType, Vector | StorageBufferNameType]] = []

equal = plant.compare(objective_plant)

ready_positions: dict[StationNameType, Vector] = {}

# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position


for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            for i in range(spec.model.stations.grid.size.x - 1,  x - 1, -1):
                if manipulated_plant.is_empty_coord(i, y):
                    continue
                focus_station_name = manipulated_plant.get_location_coordinates(i, y).name

                if focus_station_name in ready_positions:
                    ready_info = ready_positions.pop(focus_station_name)
                    result = manipulated_plant.move_station_to_another_coord(i, y, ready_info.x, ready_info.y)
                    sequence_list.append((result, Vector(i, y), Vector(ready_info.x, ready_info.y)))
                    # Update the ready positions with the station at the right of the just moved station if any
                    if ready_info.x + 1 < spec.model.stations.grid.size.x:
                        
                        objective_station = objective_plant.get_location_coordinates(ready_info.x + 1, ready_info.y)
                        if objective_station is not None:
                            ready_positions[objective_station.name] = Vector(ready_info.x + 1, ready_info.y)

                else:
                    result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
                    sequence_list.append((result[0], Vector(i, y), f"S{result[1]}"))
            
            ready_positions[objective_plant.get_location_coordinates(i, y).name] = Vector(i, y)
            break


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.compare(objective_plant)

for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:
            objective_station_name = objective_plant.get_location_coordinates(x, y).name
            result = manipulated_plant.move_station_from_buffer_to_coord(objective_station_name, x, y)
            sequence_list.append((objective_station_name, f"S{result}", Vector(x, y)))

print_results()

Manipulated Plant final version
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|                 |        0        |        1        |        2        |        3        |        4        |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|        0        |       None      |       None      |      InOut      |       None      |       None      |
|        1        |       None      |      Press      |      Robot1     |       None      |       None      |
|        2        |       None      |      Robot2     |   PartsStorage  |       None      |       None      |
|        3        |       None      |       None      |       None      |       None      |       None      |
|        4        |       None      |       None      |       None      |       None      |       None      |
+-----------------+-----------------+-----------------+-----------------+---------------